In [1]:
# Import necessary libraries for Dash app and database interaction
from dash import Dash, dcc, html, dash_table
import dash_leaflet as dl
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64
from animal_shelter import AnimalShelter

# Database connection details
username = "aacuser"
password = "newpassword"
mongo_host = "nv-desktop-services.apporto.com"
mongo_port = 30316

# Initialize connection to MongoDB via CRUD Module
db = AnimalShelter(username, password, mongo_host, mongo_port, 'AAC', 'shelter_outcomes')

# Read initial data from the database and convert to DataFrame
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)  # Remove the '_id' column for cleaner display

# Ensure there is only one instance of the app
if 'app' not in locals():
    app = Dash(__name__)

    # Encode the Grazioso Salvare logo image for embedding
    image_filename = 'Logo.png'  # Replace with your own image
    encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

    # Define the layout of the dashboard
    app.layout = html.Div([
        # Title and logo
        html.Center(html.B(html.H1('CS-340 Dashboard'))),
        html.Img(src=f'data:image/png;base64,{encoded_image}'),
        html.H3('Developed by David Hughes'),
        html.Hr(),  # Horizontal line for separation
        
        # Radio items for selecting rescue type
        html.Div([
            html.Label('Select Rescue Type:'),
            dcc.RadioItems(
                id='filter-type',
                options=[
                    {'label': 'Water Rescue', 'value': 'water'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                    {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                    {'label': 'Reset', 'value': 'reset'}
                ],
                value='reset',
                labelStyle={'display': 'inline-block'}
            )
        ]),
        html.Hr(),  # Horizontal line for separation

        # Data table to display the filtered data
        dash_table.DataTable(
            id='datatable-id',
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
            data=df.to_dict('records'),
            page_size=10,
            row_selectable='single',
            selected_rows=[],
            style_table={'overflowX': 'auto'},
            sort_action='native',
            filter_action='native',
            page_action='native',
        ),
        html.Br(),
        html.Hr(),

        # Placeholders for the breed distribution graph and map
        html.Div(className='row', style={'display': 'flex'}, children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id', className='col s12 m6')
        ])
    ])

    # Callback to update the data table based on the selected filter type
    @app.callback(
        Output('datatable-id', 'data'),
        Input('filter-type', 'value')
    )
    def update_dashboard(filter_type):
        query = {}
        # Define query based on the selected filter type
        if filter_type == 'water':
            query = {
                "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
            }
        elif filter_type == 'mountain':
            query = {
                "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
            }
        elif filter_type == 'disaster':
            query = {
                "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
            }
        elif filter_type == 'reset':
            query = {}

        # Fetch and filter data based on query
        filtered_data = db.read(query)
        filtered_df = pd.DataFrame.from_records(filtered_data)
        if not filtered_df.empty:
            filtered_df.drop(columns=['_id'], inplace=True)
        
        return filtered_df.to_dict('records')

    # Callback to update the breed distribution graph based on the data in the table
    @app.callback(
        Output('graph-id', "children"),
        Input('datatable-id', "derived_virtual_data")
    )
    def update_graphs(viewData):
        if viewData is None or len(viewData) == 0:
            return []
        
        dff = pd.DataFrame.from_dict(viewData)
        
        return [
            dcc.Graph(
                figure=px.pie(dff, names='breed', title='Breed Distribution')
            )
        ]

    # Callback to highlight selected columns in the data table
    @app.callback(
        Output('datatable-id', 'style_data_conditional'),
        Input('datatable-id', 'selected_columns')
    )
    def update_styles(selected_columns):
        if selected_columns is None:
            return []
        return [{
            'if': {'column_id': i},
            'background_color': '#D2F3FF'
        } for i in selected_columns]

    # Callback to update the map based on the selected row in the data table
    @app.callback(
        Output('map-id', "children"),
        [Input('datatable-id', "derived_virtual_data"),
         Input('datatable-id', "selected_rows")]
    )
    def update_map(viewData, selected_rows):
        if viewData is None or len(selected_rows) == 0:
            return []
        
        dff = pd.DataFrame.from_dict(viewData)
        row = selected_rows[0]
        
        try:
            # Extract latitude, longitude, breed, and name for the selected row
            latitude = dff.iloc[row]['location_lat']
            longitude = dff.iloc[row]['location_long']
            breed = dff.iloc[row]['breed']
            name = dff.iloc[row]['name']
        except (IndexError, KeyError) as e:
            print(f"Error accessing selected row: {e}")
            return []

        # Update the map with the selected row's location and information
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[latitude, longitude], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[latitude, longitude], children=[
                    dl.Tooltip(breed),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(name)
                    ])
                ])
            ])
        ]

# Run the app on a different port
app.run_server(port=8051, mode="inline")
